In [ ]:
!pip install -U hopsworks 

In [ ]:
!pip install dataframe_image

In [3]:
!pip install -q streamlit

In [ ]:
!pip install --upgrade protobuf

In [ ]:
!pip install streamlit_folium

In [ ]:
!pip install pyngrok

In [ ]:
!pip install python-dotenv

In [8]:
from pyngrok import ngrok

In [9]:
ngrok.set_auth_token("2Jzz5UrcRRGTHRB1KeKe4M38bwG_mDhqRzgG61co9WY6dMVG")

In [10]:
import hopsworks
project = hopsworks.login()


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5315


In [11]:
!git clone 'https://github.com/reyrobs/Scalable_ML_Project_AQ.git'

fatal: destination path 'Scalable_ML_Project_AQ' already exists and is not an empty directory.


In [35]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=b265fa35b98103496b70293ecac83abfbc82afea3181d0de71b86770fcd566f2
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [54]:
%%writefile app.py
import streamlit as st
import hopsworks
import os
import joblib
import pandas as pd
import dataframe_image as dfi
import streamlit as st
import joblib
import numpy as np
import folium
from streamlit_folium import st_folium, folium_static
import json
import time
from datetime import timedelta, datetime
from branca.element import Figure
from urllib.request import urlopen
import joblib
from API_call_visual_crossing import *
from API_call_AQI  import *
#----------------------------------HEADER + TITLE
def fancy_header(text, font_size=24):
    res = f'<span style="color:#ff5f27; font-size: {font_size}px;">{text}</span>'
    st.markdown(res, unsafe_allow_html=True )
st.title('⛅️Air Quality Prediction Project🌩')

progress_bar = st.sidebar.header('⚙️ Working Progress')
progress_bar = st.sidebar.progress(0)
st.write(36 * "-")

#------------------------------------  Getting Data

fancy_header('\n☁️ Getting data for predict ...')
city = 'Paris'
date = ''


WEATHER_API_KEY ="V7VHELUHHCY7JPQ8332XPR8A9"# = os.getenv('WEATHER_API_KEY')

df_weather = get_weather_df(city, date, WEATHER_API_KEY)
next_7_days = df_weather["date"]
data_7_days = df_weather.drop('date', axis=1)


model = joblib.load(urlopen('https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/aqi_model.pkl'))
progress_bar.progress(20)
st.write(36 * "-")

#------------------------------------ PREDICTING
fancy_header('\n ✔️Predicting...')

pred_7_days = model.predict(data_7_days)

df = pd.DataFrame(data=pred_7_days, index=next_7_days, columns=[f"AQI Predictions for the next 7 days"], dtype=int)

progress_bar.progress(60)
#----------------------------------

st.write(36 * "-")
fancy_header(f"🗺 Processing the map...")

fig = Figure(width=550,height=350)

my_map = folium.Map(location=[58, 20], zoom_start=3.71)
fig.add_child(my_map)
folium.TileLayer('Stamen Terrain').add_to(my_map)
folium.TileLayer('Stamen Toner').add_to(my_map)
folium.TileLayer('Stamen Water Color').add_to(my_map)
folium.TileLayer('cartodbpositron').add_to(my_map)
folium.TileLayer('cartodbdark_matter').add_to(my_map)
folium.LayerControl().add_to(my_map)

#Determining Paris
coords = [48.862, 2.346]
#-----
city = "Pris"
country="France"
text="AQI "

#-----
pred_7_days = np.asarray(pred_7_days, dtype = 'int')
#pred_7_days.round(decimals=1, out= pred_7_days)
text +=str((pred_7_days[0]))
#-----

folium.Marker(
        location=coords, popup=text, tooltip=f"<strong>{city}</strong>"
    ).add_to(my_map)


# call to render Folium map in Streamlit
folium_static(my_map)
progress_bar.progress(80)
st.sidebar.write("-" * 36)


st.sidebar.write(df)
progress_bar.progress(100)
st.button("Re-run")


Overwriting app.py


In [14]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://4f07-35-190-152-16.ngrok.io" -> "http://localhost:80"
